## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [75]:
import numpy as np
import os
from random import shuffle
import re
from collections import *
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

In [76]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [77]:
import urllib.request
import zipfile
import lxml.etree

In [78]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [79]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [80]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [81]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [82]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [83]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [84]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [85]:
len(sentences_ted)

266694

In [86]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [87]:
cnt = Counter()
for sentence in sentences_ted:
    for word in sentence:
        cnt[word] += 1
        
counts_ted_top1000 = []
for common_word in cnt.most_common(1000):
    counts_ted_top1000.append(common_word[1])

Plot distribution of top-1000 words

In [88]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [89]:
model_ted = Word2Vec(sentences_ted, min_count=10)

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [90]:
model_ted.most_similar("man")

[('woman', 0.8327457904815674),
 ('guy', 0.8038840293884277),
 ('boy', 0.7492162585258484),
 ('lady', 0.7428460717201233),
 ('girl', 0.7419350743293762),
 ('soldier', 0.7121762037277222),
 ('gentleman', 0.7038803100585938),
 ('kid', 0.690717339515686),
 ('poet', 0.6856828927993774),
 ('rabbi', 0.6648629903793335)]

In [96]:
model_ted.most_similar("woman")

[('girl', 0.8389654755592346),
 ('man', 0.8327457904815674),
 ('boy', 0.8142095804214478),
 ('lady', 0.7715435028076172),
 ('child', 0.7595261335372925),
 ('soldier', 0.748707115650177),
 ('kid', 0.7337332367897034),
 ('gentleman', 0.7168759107589722),
 ('guy', 0.7017260789871216),
 ('poet', 0.6941766738891602)]

In [97]:
model_ted.most_similar("dream")

[('legacy', 0.681867241859436),
 ('son', 0.6359061598777771),
 ('client', 0.6248802542686462),
 ('hero', 0.6058573722839355),
 ('career', 0.6047384142875671),
 ('daughter', 0.6018286347389221),
 ('faith', 0.5973504781723022),
 ('family', 0.5964661240577698),
 ('goal', 0.5922276973724365),
 ('partner', 0.5919524431228638)]

In [109]:
model_ted.most_similar("marriage")

[('equality', 0.6449852585792542),
 ('status', 0.6320967078208923),
 ('attorney', 0.6296111345291138),
 ('condition', 0.6133536696434021),
 ('religion', 0.609982430934906),
 ('instinct', 0.6051177382469177),
 ('diagnosis', 0.6047748327255249),
 ('opinion', 0.6020995378494263),
 ('faith', 0.5978935360908508),
 ('torture', 0.5932691097259521)]

In [111]:
model_ted.most_similar("technology")

[('knowledge', 0.7105307579040527),
 ('design', 0.7031307220458984),
 ('biology', 0.7024695873260498),
 ('content', 0.6812153458595276),
 ('tool', 0.6803750991821289),
 ('technologies', 0.6777361035346985),
 ('digital', 0.6713321208953857),
 ('method', 0.6664557456970215),
 ('process', 0.6631579399108887),
 ('innovation', 0.657585620880127)]

In [112]:
model_ted.most_similar("entertainment")

[('communications', 0.7452797889709473),
 ('educational', 0.7452716827392578),
 ('enterprise', 0.7433018088340759),
 ('ecology', 0.7381998300552368),
 ('journalism', 0.7338690757751465),
 ('engineering', 0.7319249510765076),
 ('sciences', 0.7281091809272766),
 ('computing', 0.7280259728431702),
 ('humanities', 0.7270147800445557),
 ('robotics', 0.7255637645721436)]

In [113]:
model_ted.most_similar("design")

[('technology', 0.7031307220458984),
 ('biology', 0.6955777406692505),
 ('science', 0.6780058145523071),
 ('mathematics', 0.6582353115081787),
 ('engineering', 0.6562415957450867),
 ('mathematical', 0.6546720266342163),
 ('tool', 0.6542975902557373),
 ('model', 0.648714542388916),
 ('art', 0.6454150676727295),
 ('architecture', 0.6380443572998047)]

In [105]:
print(model_ted.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])[:3])
print(model_ted.most_similar_cosmul(positive=['computer', 'mind'], negative=['machine'])[:3])
print(model_ted.most_similar_cosmul(positive=['life','meaning'])[:3])
print(model_ted.most_similar_cosmul(positive=['war', 'usa'])[:3])

[('james', 0.9280941486358643), ('queen', 0.9276872277259827), ('president', 0.9250240325927734)]
[('thoughts', 0.904100239276886), ('hands', 0.8828719258308411), ('pocket', 0.8704074621200562)]
[('happiness', 0.649143636226654), ('humanity', 0.6329495906829834), ('experience', 0.6264689564704895)]
[('vietnam', 0.7212347984313965), ('congo', 0.7053382396697998), ('syria', 0.7051510810852051)]


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [93]:
words_top_ted = []
for common_word in cnt.most_common(1000):
    words_top_ted.append(common_word[0])

# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_vec_ted = model_ted[words_top_ted]

In [94]:
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [95]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Extra: Literatura

In [4]:
with open('/home/matheus/Documents/litearatura/copilados', 'r') as in_f:
    liter_text = in_f.read().replace('!','.').replace('?','.').lower().split('.')

liter_text[:3]

['\x0c1\n\nnao se sabe se kublai khan acredita em tudo o que diz marco polo quando este lhe\ndescreve as cidades visitadas em suas missoes diplomaticas, mas o imperador dos\ntartaros certamente continua a ouvir o jovem veneziano com maior curiosidade e\natencao do que a qualquer outro de seus enviados ou exploradores',
 ' existe um momento\nna vida dos imperadores que se segue ao orgulho pela imensa amplitude dos territorios\nque conquistamos, a melancolia e ao alivio de saber que em breve desistiremos de\nconhece-los e compreende-los, uma sensacao de vazio que surge ao calar da noite com\no odor dos elefantes apos a chuva e das cinzas de sandalo que se resfriam nos braseiros,\numa vertigem que faz estremecer os rios e as montanhas historiadas nos fulvos dorsos\ndos planisferios, enrolando um depois do outro os despachos que anunciam o\naniquilamento dos ultimos exercitos inimigos de derrota em derrota, e abrindo o lacre\ndos sinetes de reis dos quais nunca se ouviu falar e que implora

In [5]:
for s_i in range(len(liter_text)):
    liter_text[s_i] = re.sub("[^a-z]", " ", liter_text[s_i]).split()

liter_text[:2]

[['nao',
  'se',
  'sabe',
  'se',
  'kublai',
  'khan',
  'acredita',
  'em',
  'tudo',
  'o',
  'que',
  'diz',
  'marco',
  'polo',
  'quando',
  'este',
  'lhe',
  'descreve',
  'as',
  'cidades',
  'visitadas',
  'em',
  'suas',
  'missoes',
  'diplomaticas',
  'mas',
  'o',
  'imperador',
  'dos',
  'tartaros',
  'certamente',
  'continua',
  'a',
  'ouvir',
  'o',
  'jovem',
  'veneziano',
  'com',
  'maior',
  'curiosidade',
  'e',
  'atencao',
  'do',
  'que',
  'a',
  'qualquer',
  'outro',
  'de',
  'seus',
  'enviados',
  'ou',
  'exploradores'],
 ['existe',
  'um',
  'momento',
  'na',
  'vida',
  'dos',
  'imperadores',
  'que',
  'se',
  'segue',
  'ao',
  'orgulho',
  'pela',
  'imensa',
  'amplitude',
  'dos',
  'territorios',
  'que',
  'conquistamos',
  'a',
  'melancolia',
  'e',
  'ao',
  'alivio',
  'de',
  'saber',
  'que',
  'em',
  'breve',
  'desistiremos',
  'de',
  'conhece',
  'los',
  'e',
  'compreende',
  'los',
  'uma',
  'sensacao',
  'de',
  'vazio',


In [6]:
shuffle(liter_text)
print(len(liter_text))

296415


In [7]:
model_liter = Word2Vec(liter_text, min_count=5)

In [8]:
model_liter.most_similar("homem")

[('animal', 0.7381560206413269),
 ('rapaz', 0.7013584971427917),
 ('sujeito', 0.652496874332428),
 ('mendigo', 0.6234924793243408),
 ('individuo', 0.6079890727996826),
 ('pintor', 0.5963858366012573),
 ('fantasma', 0.5868741273880005),
 ('ladrao', 0.5868232250213623),
 ('assassino', 0.5821999311447144),
 ('cavalheiro', 0.5781742334365845)]

In [9]:
model_liter.most_similar("mulher")

[('moca', 0.7682394981384277),
 ('menina', 0.7588468790054321),
 ('crianca', 0.7381477355957031),
 ('filha', 0.7297366261482239),
 ('mae', 0.6704534292221069),
 ('criada', 0.6627806425094604),
 ('pessoa', 0.6523498296737671),
 ('viuva', 0.6336188912391663),
 ('rapariga', 0.6304502487182617),
 ('irma', 0.6278578042984009)]

In [38]:
model_liter.most_similar("crime")

[('disparate', 0.4797269403934479),
 ('erro', 0.4602298438549042),
 ('castigo', 0.4577246606349945),
 ('pecado', 0.45394080877304077),
 ('ato', 0.43664786219596863),
 ('absurdo', 0.43515482544898987),
 ('assunto', 0.42969661951065063),
 ('pensamento', 0.4277363419532776),
 ('rodka', 0.4204072058200836),
 ('acto', 0.4183739125728607)]

In [14]:
model_liter.most_similar("sonho")

[('pensamento', 0.7465449571609497),
 ('delirio', 0.7224061489105225),
 ('poema', 0.7125514149665833),
 ('ato', 0.7078797221183777),
 ('discurso', 0.6949011087417603),
 ('erro', 0.6935181617736816),
 ('milionesimo', 0.6853308081626892),
 ('pesadelo', 0.6822788715362549),
 ('sono', 0.6689560413360596),
 ('episodio', 0.667937695980072)]

In [35]:
model_liter.most_similar("amor")

[('odio', 0.7416133880615234),
 ('coracao', 0.7045820951461792),
 ('espirito', 0.6983347535133362),
 ('sofrimento', 0.6909886598587036),
 ('temor', 0.6888531446456909),
 ('sentimento', 0.6749279499053955),
 ('interesse', 0.6704716086387634),
 ('suicidio', 0.6636437773704529),
 ('futuro', 0.6622185707092285),
 ('remorso', 0.6566768288612366)]

In [54]:
model_liter.most_similar("morte")

[('ausencia', 0.6735057830810547),
 ('vida', 0.6700690984725952),
 ('doenca', 0.6693675518035889),
 ('cegueira', 0.654128909111023),
 ('memoria', 0.6272670030593872),
 ('existencia', 0.6218945384025574),
 ('gloria', 0.6178592443466187),
 ('guerra', 0.6163017153739929),
 ('patria', 0.6059645414352417),
 ('culpa', 0.6047093272209167)]

In [69]:
print(model_liter.most_similar_cosmul(positive=['roma', 'franca'], negative=['italia']), '\n')
print(model_liter.most_similar_cosmul(positive=['barco'], negative=['navegar']), '\n')
print(model_liter.most_similar_cosmul(positive=['guerra', 'paz']), '\n')
print(model_liter.most_similar_cosmul(positive=['homem'], negative=['deus']), '\n')
print(model_liter.most_similar_cosmul(positive=['labirinto', 'xadrez']), '\n')

[('napoleao', 0.983713686466217), ('paris', 0.9494947791099548), ('julho', 0.937974214553833), ('cesar', 0.9322437644004822), ('luis', 0.9317600131034851), ('bonaparte', 0.927058219909668), ('herat', 0.9149861931800842), ('avinhao', 0.9110685586929321), ('angouleme', 0.9056127071380615), ('guerra', 0.9046072363853455)] 

[('quarto', 1.5500776767730713), ('carro', 1.535387396812439), ('jeito', 1.5226939916610718), ('espelho', 1.5090110301971436), ('banco', 1.5057342052459717), ('buraco', 1.5042953491210938), ('relogio', 1.5012691020965576), ('dedo', 1.4911432266235352), ('olho', 1.4834256172180176), ('cadaver', 1.4717621803283691)] 

[('espanha', 0.6328234076499939), ('gloria', 0.6323890686035156), ('inglaterra', 0.6265755891799927), ('familia', 0.6252608299255371), ('patria', 0.6241098642349243), ('execucao', 0.6170610785484314), ('europa', 0.6161503791809082), ('franca', 0.6156556010246277), ('italia', 0.6097943782806396), ('ilha', 0.6090291142463684)] 

[('parecia', 1.612980127334594

In [70]:
cnt = Counter()
for sentence in liter_text:
    for word in sentence:
        cnt[word] += 1
        
counts_liter_top1000 = []
for common_word in cnt.most_common(1000):
    counts_liter_top1000.append(common_word[0])
    
words_top_vec_liter = model_liter[counts_liter_top1000]

In [73]:
tsne = TSNE(n_components=2, random_state=0)
words_top_liter_tsne = tsne.fit_transform(words_top_vec_liter)

In [74]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_liter_tsne[:,0],
                                    x2=words_top_liter_tsne[:,1],
                                    names=counts_liter_top1000))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [ ]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [ ]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [ ]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)

for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split()
    
del input_text 

In [ ]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [ ]:
cnt = Counter()
for sentence in sentences_wiki:
    for word in sentence:
        cnt[word] += 1
        
counts_wiki_top1000 = []
for common_word in cnt.most_common(1000):
    counts_wiki_top1000.append(common_word[1])

In [ ]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [ ]:
model_wiki = Word2Vec(sentences_wiki, min_count=10)

In [ ]:
model_wiki.most_similar("man")

In [ ]:
model_wiki.most_similar("computer")

In [ ]:
words_top_wiki = []
for common_word in cnt.most_common(1000):
    words_top_wiki.append(common_word[0])


#### t-SNE visualization

In [ ]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)